In [ ]:
import os

import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

from src.data import raw_io as io
from src.data import conditions as cond

# Load raw data

In [ ]:
DATA_PATH = os.path.join('..', 'data', 'processed', 'trials_by_celltype')

In [ ]:
def load_sessions_by_day(day: cond.Day):
    sessions = {}
    with h5py.File(os.path.join(DATA_PATH, str(day) + '.h5'), 'r') as f:
        for cell_type in cond.CellType:
            sessions[cell_type] = []
            for mouse in f[cell_type.as_path()].keys():
                sessions[cell_type].append(
                    io.SessionTrials.load(f[cell_type.as_path()][mouse])
                )

        f.close()
    
    return sessions

In [ ]:
raw_data_by_day = {
    cond.DAYS[0]: load_sessions_by_day(cond.DAYS[0]), 
    cond.DAYS[-1]: load_sessions_by_day(cond.DAYS[-1])
}

In [ ]:
def get_mean_fluorescence_by_cell_type(daily_data):
    mean_fluorescence = {}
    for cell_type in daily_data.keys():

        # Concatenate fluorescence of each mouse
        mean_fluorescence[cell_type] = daily_data[cell_type][0].fluo.trial_mean()
        for i in range(1, len(daily_data[cell_type])):
            trial_mean_fluo = daily_data[cell_type][i].fluo.trial_mean()
            mean_fluorescence[cell_type].append(
                trial_mean_fluo
            )
            del trial_mean_fluo

        print('Got {} {} cells'.format(mean_fluorescence[cell_type].fluo.shape[0], cell_type))
        mean_fluorescence[cell_type].remove_nan()
    
    return mean_fluorescence

In [ ]:
mean_fluorescence_by_day = {
    day: get_mean_fluorescence_by_cell_type(raw_data_by_day[day]) 
    for day in raw_data_by_day.keys()
}

# Heatmaps

In [ ]:
def plot_heatmaps_by_celltype(daily_mean_data):
    for cell_type in cond.CellType:
        print('Clustering mean {} responses'.format(cell_type))
        g = sns.clustermap(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo, col_cluster=False, method='ward')
        g.ax_heatmap.set_xticks(np.linspace(0, mean_fluorescence_by_day[cond.DAYS[0]][cell_type].num_frames, 10))
        g.ax_heatmap.set_xticklabels([
            '{:.1f}'.format(i) for i in
            np.linspace(0, mean_fluorescence_by_day[cond.DAYS[0]][cell_type].duration, 10) - 2.0
        ])
        g.ax_heatmap.set_xlabel('Time from trial start (s)')
        g.ax_heatmap.set_yticks([])
        plt.show()

In [ ]:
for cell_type in cond.CellType:
    print('Clustering mean {} responses'.format(cell_type))
    g = sns.clustermap(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo, col_cluster=False, method='ward')
    g.ax_heatmap.set_xticks(np.linspace(0, mean_fluorescence_by_day[cond.DAYS[0]][cell_type].num_frames, 10))
    g.ax_heatmap.set_xticklabels([
        '{:.1f}'.format(i) for i in
        np.linspace(0, mean_fluorescence_by_day[cond.DAYS[0]][cell_type].duration, 10) - 2.0
    ])
    g.ax_heatmap.set_xlabel('Time from trial start (s)')
    g.ax_heatmap.set_yticks([])
    plt.show()

# t-SNE

## Each cell type separately

In [ ]:
for cell_type in cond.CellType:
    print('Clustering mean {} responses'.format(cell_type))
    tsne = TSNE(perplexity=20, random_state=43)
    projected = tsne.fit_transform(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo)
    
    plt.figure()
    
    plt.subplot(111)
    plt.title(str(cell_type))
    plt.plot(projected[:, 0], projected[:, 1], 'k.')
    plt.show()

## All cell types together

In [ ]:
labels = []
fluo_matrices = []
for cell_type in cond.CellType:
    labels.extend(
        [
            str(cell_type) for i in 
            range(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo.shape[0])
        ]
    )
    fluo_matrices.append(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo)
    
min_num_frames = np.min([fl.shape[1] for fl in fluo_matrices])
fluo_grand_matrix = np.concatenate([fl[:, :min_num_frames] for fl in fluo_matrices])

In [ ]:
tsne = TSNE(perplexity=20, random_state=42, init='pca')
projected = tsne.fit_transform(fluo_grand_matrix)

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(111)
sns.scatterplot(x=projected[:, 0], y=projected[:, 1], hue=labels, alpha=0.7)
plt.xticks([])
plt.yticks([])

plt.show()

### Last day

In [ ]:
labels = []
fluo_matrices = []
for cell_type in cond.CellType:
    labels.extend(
        [
            str(cell_type) for i in 
            range(mean_fluorescence_by_day[cond.DAYS[-1]][cell_type].fluo.shape[0])
        ]
    )
    fluo_matrices.append(mean_fluorescence_by_day[cond.DAYS[-1]][cell_type].fluo)
    
min_num_frames = np.min([fl.shape[1] for fl in fluo_matrices])
fluo_grand_matrix = np.concatenate([fl[:, :min_num_frames] for fl in fluo_matrices])

In [ ]:
tsne = TSNE(perplexity=20, random_state=42, init='pca')
projected = tsne.fit_transform(fluo_grand_matrix)

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(111)
sns.scatterplot(x=projected[:, 0], y=projected[:, 1], hue=labels, alpha=0.7)
plt.xticks([])
plt.yticks([])

plt.show()

## Each cell type separately

In [ ]:
tsne_args = {'perplexity': 20, 'random_state': 43, 'init': 'pca'}

for cell_type in cond.CellType:
    print('Clustering mean {} responses'.format(cell_type))
    
    plt.figure(figsize=(10, 4))
    
    # First day
    plt.subplot(131)
    plt.title(str(cell_type) + ' ' + str(cond.DAYS[0]))

    projected = TSNE(**tsne_args).fit_transform(mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo)
    sns.scatterplot(projected[:, 0], projected[:, 1])
    plt.xticks([])
    plt.xlabel('')
    plt.yticks([])
    plt.ylabel('')
    
    # Last day, only cells from first day
    plt.subplot(132)
    plt.title(str(cell_type) + ' ' + str(cond.DAYS[-1]) + '\ncells from first day')
    
    num_first_day_cells = mean_fluorescence_by_day[cond.DAYS[0]][cell_type].fluo.shape[0]
    projected = TSNE(**tsne_args).fit_transform(
        mean_fluorescence_by_day[cond.DAYS[-1]][cell_type].fluo[:num_first_day_cells, :]
    )
    sns.scatterplot(projected[:, 0], projected[:, 1])
    plt.xticks([])
    plt.xlabel('')
    plt.yticks([])
    plt.ylabel('')
    
    # Last day, all cells
    plt.subplot(133)
    plt.title(str(cell_type) + ' ' + str(cond.DAYS[-1]) + '\nall cells')
    
    projected = TSNE(**tsne_args).fit_transform(mean_fluorescence_by_day[cond.DAYS[-1]][cell_type].fluo)
    cells_from_day = [cond.DAYS[0] for i in range(num_first_day_cells)]
    cells_from_day.extend([cond.DAYS[-1] for i in range(
        mean_fluorescence_by_day[cond.DAYS[-1]][cell_type].fluo.shape[0] - num_first_day_cells
    )])
    sns.scatterplot(projected[:, 0], projected[:, 1], hue=cells_from_day)
    plt.xticks([])
    plt.xlabel('')
    plt.yticks([])
    plt.ylabel('')
    
    plt.show()